In [1]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 MB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.9/321.9 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 33.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.9 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
tor

In [2]:
from google.colab import files

uploaded = files.upload()  # Upload the model file

Saving otp_failover_best_model.pkl to otp_failover_best_model.pkl


In [6]:
import gradio as gr
import pandas as pd
import joblib
# تحميل الموديل المدرب
model_filename = list(uploaded.keys())[0]  # Get the filename
# تحميل الموديل المدرب
model_filename = "otp_failover_best_model.pkl"
best_model = joblib.load(model_filename)

# قائمة الميزات المطلوبة بناءً على `X_train.columns`
required_features = [
    "Adjusted Response Time", "Error Rate", "Primary Vendor Failure Rate",
    "Backup Vendor Success Rate", "Retry Attempts", "Network Latency (ms)",
    "Time Period"
]

# prediction
def predict_otp(adjusted_time, error_rate, primary_failure, backup_success, retries, network_latency, time_period):
    try:
        # إنشاء DataFrame للقيم المدخلة
        input_data = pd.DataFrame([{
            "Response Time(0-15 s)": float(adjusted_time),
            "Error Rate(0-0.1)": float(error_rate),
            "Primary Vendor Failure Rate (0-0.1)": float(primary_failure),
            "Backup Vendor Success Rate(0-0.1)": float(backup_success),
            "Retry Attempts(0-3)": int(retries),
            "Time Period": int(time_period)
        }])

        # التحقق من الأعمدة المفقودة وإضافة القيم الافتراضية
        missing_cols = set(required_features) - set(input_data.columns)
        for col in missing_cols:
            input_data[col] = 0

        # التأكد من ترتيب الأعمدة كما في بيانات التدريب
        input_data = input_data[required_features]

        # توقع الفشل
        prediction = best_model.predict(input_data)[0]
        return "✅ Delivered" if prediction == 0 else "🔄 Switch to Backup Vendor"

    except Exception as e:
        return f"🚨 Error: {str(e)}"

# تصميم واجهة Gradio
with gr.Blocks(theme=gr.themes.Default(), css="body {background-color: #f1eedb;} .gradio-container {max-width: 600px; margin: auto;}") as demo:
    gr.Markdown("""
    # 🔮 OTP Failover Prediction 🚀
    **Enter OTP system metrics to predict whether failover is needed.**
    """)

    with gr.Row():
        adjusted_time = gr.Number(label="Response Time(0-15s)", value=0.0)
        error_rate = gr.Number(label="Error Rate(0-0.1)", value=0.0)

    with gr.Row():
        primary_failure = gr.Number(label="Primary Vendor Failure Rate(0-0.1)", value=0.0)
        backup_success = gr.Number(label="Backup Vendor Success Rate", value=0.0)

    with gr.Row():
        retries = gr.Number(label="Retry Attempts", value=0)
        network_latency = gr.Number(label="Network Latency (100-15000 ms)", value=0.0)

    time_period = gr.Radio([0, 1], label="Time Period", choices=[("Off-Peak", 0), ("Peak", 1)], value=0)

    submit_btn = gr.Button("🔍 Predict OTP Failover")
    output_text = gr.Textbox(label="Prediction Result", interactive=False)

    submit_btn.click(predict_otp, inputs=[adjusted_time, error_rate, primary_failure, backup_success, retries, network_latency, time_period], outputs=output_text)

# تشغيل Gradio
demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://67126483bdb9d901f1.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
